In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           Var,
                           Reals,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration
from R125_bmimPF6_PR import configuration as configuration_R125
from R32_bmimPF6_PR import configuration as configuration_R32

# Load data from csv
data = pd.read_csv('HFCSIL_all.csv')

In [5]:
a = -1.601578
b = -0.111417
c = 0.26803
d = 0.02700
e = 1.5
f = 0.170299
g = -0.3153
h = -0.04879

def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    m.fs.a = Var(within=Reals, bounds=(-5,5), initialize= a)
    m.fs.b = Var(within=Reals, bounds=(-5,5), initialize= b)
    m.fs.c = Var(within=Reals, bounds=(-5,5), initialize= c)
    m.fs.d = Var(within=Reals, bounds=(-5,5), initialize= d)
    m.fs.e = Var(within=Reals, bounds=(-5,5), initialize= e)
    m.fs.f = Var(within=Reals, bounds=(-5,5), initialize= f)
    m.fs.g = Var(within=Reals, bounds=(-5,5), initialize= g)
    m.fs.h = Var(within=Reals, bounds=(-5,5), initialize= h)
    
    eps1 = 0.1
    eps2 = 0.1
    if  float(data["x_R32"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R125)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R125"])+eps1
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R125"].fix(x)


        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(a)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(b)

        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(e)
        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(f)

        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].unfix()
        
        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R125"].fix(float(data["x_R125"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].fix(float(data["x_IL"]))

        m.fs.state_block.mole_frac_comp["R125"].fix(float(data["x_R125"])+eps1)
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimPF6", "R125"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimPF6", "R125"].setub(5)

        m.fs.properties.PR_kappa_A["R125", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_A["R125", "bmimPF6"].setub(5)

        m.fs.properties.PR_kappa_B["bmimPF6", "R125"].setlb(-5)
        m.fs.properties.PR_kappa_B["bmimPF6", "R125"].setub(5)

        m.fs.properties.PR_kappa_B["R125", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_B["R125", "bmimPF6"].setub(5)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimPF6", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimPF6"])
        m.fs.E = Constraint(expr= m.fs.e == m.fs.properties.PR_kappa_B["bmimPF6", "R125"])
        m.fs.F = Constraint(expr= m.fs.f == m.fs.properties.PR_kappa_B["R125", "bmimPF6"])

    elif  float(data["x_R125"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R32)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R32"])+eps2
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R32"].fix(x)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(c)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(d)
        
        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(g)
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(h)
        
        
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].unfix()
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].unfix()   

        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R32"].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].fix(float(data["x_IL"]))

        m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"])+eps2)
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimPF6", "R32"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimPF6", "R32"].setub(5)

        m.fs.properties.PR_kappa_A["R32", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_A["R32", "bmimPF6"].setub(5)
        
        m.fs.properties.PR_kappa_B["bmimPF6", "R32"].setlb(-5)
        m.fs.properties.PR_kappa_B["bmimPF6", "R32"].setub(5)

        m.fs.properties.PR_kappa_B["R32", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_B["R32", "bmimPF6"].setub(5)
        
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimPF6", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimPF6"])
        m.fs.G = Constraint(expr= m.fs.g == m.fs.properties.PR_kappa_B["bmimPF6", "R32"])
        m.fs.H = Constraint(expr= m.fs.h == m.fs.properties.PR_kappa_B["R32", "bmimPF6"])
        
    else:

        m.fs.properties = GenericParameterBlock(default=configuration)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})

        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(298.15)
        m.fs.state_block.pressure.fix(100000)
        m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
        m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.96)

        m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
        m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
        
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(a)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(b)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(c)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(d)
        
        m.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
        m.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)

        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(e)
        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(f)
        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(g)
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(h)
        
        # Initialize the flash unit
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        # Fix at actual temperature

        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
  
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
        m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].unfix()
        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].unfix()
        
        # Set bounds on variables to be estimated

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-5)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(5)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-5)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(5)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-5)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(5)

        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-5)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(5)
        
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].setlb(-5)
        m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].setub(5)

        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].setlb(-5)
        m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].setub(5)

        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].setlb(-5)
        m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].setub(5)

        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].setlb(-5)
        m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].setub(5)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimPF6", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimPF6"])
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimPF6", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimPF6"])
        m.fs.E = Constraint(expr= m.fs.e == m.fs.properties.PR_kappa_B["bmimPF6", "R125"])
        m.fs.F = Constraint(expr= m.fs.f == m.fs.properties.PR_kappa_B["R125", "bmimPF6"])
        m.fs.G = Constraint(expr= m.fs.g == m.fs.properties.PR_kappa_B["bmimPF6", "R32"])
        m.fs.H = Constraint(expr= m.fs.h == m.fs.properties.PR_kappa_B["R32", "bmimPF6"])        
        # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.000001, "x_IL":0.7312,  
             "x_R125":0.5, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-03-16 16:30:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 4


In [7]:
variable_name = [
                "fs.a",
                "fs.b",
                "fs.c",
                "fs.d",
                "fs.e",
                "fs.f",
                "fs.g",
                "fs.h"]

In [8]:
# def SSE(m, data):
#     expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
#     return expr*1e-9

def SSE(m, data):
    if  float(data["x_R32"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9
    elif  float(data["x_R125"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9
    else:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9*10*2.62

In [9]:
solver_options = {"max_iter": 10}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2022-03-16 16:30:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:30:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:30:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:30:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:31:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:31:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:31:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:31:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:31:04 [INFO] idaes

2022-03-16 16:31:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the followin

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.491799

The values for the parameters are as follows:
fs.a = -1.409556796573872
fs.b = -0.15786019967248974
fs.c = -0.009318303807974685
fs.d = -0.024822998832604335
fs.e = 1.7569963498315664
fs.f = 0.2087715763088585
fs.g = -0.09212392740461854
fs.h = -0.005542191381908365


In [11]:
print(large_residuals_set(pest.ef_instance))
# print(degrees_of_freedom(pest.ef_instance))

ComponentSet([])


In [12]:
# fs.properties.PR_kappa_A[R32,bmimPF6] = -0.03213185866632573
# fs.properties.PR_kappa_A[bmimPF6,R32] = -0.15261951215083822
# fs.properties.PR_kappa_A[bmimPF6,R125] = 0.4104779938348984
# fs.properties.PR_kappa_A[R125,bmimPF6] = 0.05514778922730282

In [13]:
from idaes.core.util.model_statistics import report_statistics
print(report_statistics(pest.ef_instance))


Model Statistics  -  _Q_opt 

Degrees of Freedom: 14 

Total No. Variables: 4188 
    No. Fixed Variables: 2484
    No. Unused Variables: 1089 (Fixed):1089)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 1690 
    No. Equality Constraints: 1690 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 70 (Deactivated: 69)

No. Blocks: 561 (Deactivated: 0) 
No. Expressions: 2475 

None
